In [3]:
import logging
from naslib.defaults.trainer import Trainer
from naslib.optimizers import DARTSOptimizer
from naslib.search_spaces import DartsSearchSpace
from naslib.utils import utils, setup_logger, get_config_from_args, set_seed, log_args
from naslib.search_spaces.core.graph import Graph, EdgeData
from naslib.search_spaces.core import primitives as ops
from torch import nn
from fvcore.common.config import CfgNode
from copy import deepcopy

In [4]:
config = utils.get_config_from_args(config_type='nas')
# config.search.epochs = 1 # for testing
config.optimizer = 'darts'
config.save = '{}/{}/{}/{}'.format(config.out_dir, config.dataset, config.optimizer, config.seed)
utils.set_seed(config.seed)
utils.log_args(config)


logger = setup_logger(config.save + '/log.log')
logger.setLevel(logging.INFO)

Namespace(config_file='/project/dl2022s/robertsj/NASLib/naslib/benchmarks/nas_predictors/discrete_config.yaml', dist_backend='nccl', dist_url='tcp://127.0.0.1:8888', eval_only=False, gpu=None, model_path=None, multiprocessing_distributed=False, opts=[], rank=0, resume=False, seed=0, world_size=1)


In [3]:
from naslib.search_spaces.core.graph import Graph, EdgeData
from naslib.search_spaces.core import primitives as ops
from torch import nn
from copy import deepcopy

class DartsSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'a_stage_2', 
        'a_stage_3'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        channels = [(16 * 5 * 5, 120), (120, 84), (84, 10)]
        stages = ['a_stage_1', 'a_stage_2', 'a_stage_3']

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2) # intermediate node
        activation_cell.add_node(3) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData())]) # mutable intermediate edge
        activation_cell.edges[1, 2].set('cell_name', 'activation_cell') 
        activation_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable output edge

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        for i, scope in zip(range(3, 6), stages):
            self.add_node(i, subgraph=deepcopy(activation_cell).set_scope(scope).set_input([i-1])) # activation node i
            self.nodes[i]['subgraph'].name = scope
        self.add_node(6) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 6)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        
        for scope, (in_dim, out_dim) in zip(stages, channels):
            self.update_edges(
                update_func=lambda edge: self._set_ops(edge, in_dim, out_dim),
                scope=scope,
                private_edge_data=True,
            )

    def _set_ops(self, edge, in_dim, out_dim):
        if out_dim != 10:
            edge.data.set('op', [
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.ReLU()),
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.Hardswish()),
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.LeakyReLU()),
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.Identity())
            ])
        else:
            edge.data.set('op', [
                ops.Sequential(nn.Linear(in_dim, out_dim), nn.Softmax(dim=1))
            ])  

In [5]:
search_space = DartsSearchSpace()

WARNING [06/27 09:31:39 nl.search_spaces.core.graph]: Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`
WARNING [06/27 09:31:39 nl.search_spaces.core.graph]: Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`
WARNING [06/27 09:31:39 nl.search_spaces.core.graph]: Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`


In [6]:
optimizer = DARTSOptimizer(config)
optimizer.adapt_search_space(search_space)

WARNING [06/27 09:31:42 nl.search_spaces.core.graph]: Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`
WARNING [06/27 09:31:42 nl.search_spaces.core.graph]: Update function could not be veryfied. Be cautious with the setting of `private_edge_data` in `update_edges()`
[06/27 09:31:42 nl.optimizers.oneshot.darts.optimizer]: Parsed graph:
Graph normal_cell:
 Graph(
  (normal_cell-edge(1,3)): MixedOp(
    (primitive-0): Identity()
    (primitive-1): Zero (stride=1)
    (primitive-2): MaxPool(
      (maxpool): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    )
    (primitive-3): AvgPool(
      (avgpool): AvgPool2d(kernel_size=3, stride=1, padding=1)
    )
    (primitive-4): SepConv(
      (op): Sequential(
        (0): ReLU()
        (1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (2): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [6]:
trainer = Trainer(optimizer, config)
trainer.search()

[06/25 15:24:40 nl.defaults.trainer]: param size = 0.236858MB
[06/25 15:24:40 nl.defaults.trainer]: Start training
Files already downloaded and verified
Files already downloaded and verified
[06/25 15:24:43 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.000529, +0.001313, +0.000432, +0.000184, 1
+0.000824, -0.001669, +0.000387, +0.000960, 3
+0.001344, 0
WARNING [06/25 15:24:44 nl.search_spaces.core.graph]: Comb_op is ignored if subgraph is defined!
[06/25 15:24:44 nl.defaults.trainer]: Epoch 0-0, Train loss: 2.30296, validation loss: 2.30231, learning rate: [0.025]
[06/25 15:24:44 nl.defaults.trainer]: cuda consumption
 |===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===================

[06/25 15:24:49 nl.defaults.trainer]: Epoch 0-30, Train loss: 2.30282, validation loss: 2.30232, learning rate: [0.025]
[06/25 15:24:54 nl.defaults.trainer]: Epoch 0-60, Train loss: 2.30240, validation loss: 2.30287, learning rate: [0.025]
[06/25 15:24:59 nl.defaults.trainer]: Epoch 0-90, Train loss: 2.30253, validation loss: 2.30225, learning rate: [0.025]
[06/25 15:25:04 nl.defaults.trainer]: Epoch 0-120, Train loss: 2.30295, validation loss: 2.30237, learning rate: [0.025]
[06/25 15:25:07 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([-0.0119,  0.0039, -0.0163,  0.0211], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([ 0.0005,  0.0131, -0.0039, -0.0062], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-5.3323e-24], device='cuda:0', requires_grad=True)]
[06/25 15:25:07 nl.defaults.trainer]: Epoch 0 done. Train accuracy (top1, top5): 9.88286, 49.89429, Validation accuracy:

[06/25 15:27:04 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.323537, -0.328100, -0.350614, +0.428123, 3
-0.255544, -0.262764, -0.255105, +0.330919, 3
+0.000000, 0
[06/25 15:27:06 nl.defaults.trainer]: Epoch 6-13, Train loss: 2.16771, validation loss: 2.16863, learning rate: [0.02478744700874427]
[06/25 15:27:11 nl.defaults.trainer]: Epoch 6-43, Train loss: 2.16364, validation loss: 2.16331, learning rate: [0.02478744700874427]
[06/25 15:27:16 nl.defaults.trainer]: Epoch 6-73, Train loss: 2.15727, validation loss: 2.13947, learning rate: [0.02478744700874427]
[06/25 15:27:21 nl.defaults.trainer]: Epoch 6-103, Train loss: 2.17639, validation loss: 2.13943, learning rate: [0.02478744700874427]
[06/25 15:27:26 nl.defaults.trainer]: Epoch 6-133, Train loss: 2.13354, validation loss: 2.15798, learning rate: [0.02478744700874427]
[06/25 15:27:27 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([

[06/25 15:29:24 nl.defaults.trainer]: Epoch 11 done. Train accuracy (top1, top5): 35.25429, 81.03429, Validation accuracy: 34.34934, 80.16081
[06/25 15:29:24 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.290776, -0.272266, -0.317259, +0.409648, 3
-0.275819, -0.262987, -0.221904, +0.351612, 3
+0.000000, 0
[06/25 15:29:28 nl.defaults.trainer]: Epoch 12-26, Train loss: 2.12649, validation loss: 2.14147, learning rate: [0.02415731783065902]
[06/25 15:29:33 nl.defaults.trainer]: Epoch 12-56, Train loss: 2.04177, validation loss: 2.09675, learning rate: [0.02415731783065902]
[06/25 15:29:38 nl.defaults.trainer]: Epoch 12-85, Train loss: 2.15215, validation loss: 2.08459, learning rate: [0.02415731783065902]
[06/25 15:29:44 nl.defaults.trainer]: Epoch 12-115, Train loss: 2.09759, validation loss: 2.07379, learning rate: [0.02415731783065902]
[06/25 15:29:47 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containin

[06/25 15:31:45 nl.defaults.trainer]: Epoch 17 done. Train accuracy (top1, top5): 40.64571, 86.45714, Validation accuracy: 40.61930, 85.94035
[06/25 15:31:45 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.229880, -0.213105, -0.278955, +0.359372, 3
-0.275161, -0.214319, -0.215124, +0.344731, 3
+0.000000, 0
[06/25 15:31:46 nl.defaults.trainer]: Epoch 18-3, Train loss: 2.03039, validation loss: 2.01571, learning rate: [0.023131935106024185]
[06/25 15:31:51 nl.defaults.trainer]: Epoch 18-33, Train loss: 2.01263, validation loss: 2.04525, learning rate: [0.023131935106024185]
[06/25 15:31:56 nl.defaults.trainer]: Epoch 18-62, Train loss: 2.04031, validation loss: 2.05108, learning rate: [0.023131935106024185]
[06/25 15:32:01 nl.defaults.trainer]: Epoch 18-92, Train loss: 2.02848, validation loss: 2.03425, learning rate: [0.023131935106024185]
[06/25 15:32:06 nl.defaults.trainer]: Epoch 18-122, Train loss: 2.02320, validation loss: 2.02833, learning ra

...

[06/25 15:56:46 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([ 0.1223,  0.1131, -0.0561, -0.0004], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-0.2620, -0.1715, -0.3374,  0.4341], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([1.1911e-43], device='cuda:0', requires_grad=True)]
[06/25 15:56:46 nl.defaults.trainer]: Epoch 81 done. Train accuracy (top1, top5): 61.30571, 94.39714, Validation accuracy: 59.12409, 94.06079
[06/25 15:56:46 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.122254, +0.113092, -0.056064, -0.000362, 0
-0.262021, -0.171534, -0.337417, +0.434138, 3
+0.000000, 0
[06/25 15:56:49 nl.defaults.trainer]: Epoch 82-13, Train loss: 1.83856, validation loss: 1.86192, learning rate: [0.002868064893975819]
[06/25 15:56:54 nl.defaults.trainer]: Epoch 82-43, Train loss: 1.82248, validation loss: 1.87725, learning rate: [0.002

[06/25 15:58:56 nl.defaults.trainer]: Epoch 87-76, Train loss: 1.84948, validation loss: 1.84514, learning rate: [0.0019869444917922276]
[06/25 15:59:01 nl.defaults.trainer]: Epoch 87-106, Train loss: 1.82180, validation loss: 1.85563, learning rate: [0.0019869444917922276]
[06/25 15:59:06 nl.defaults.trainer]: Epoch 87-136, Train loss: 1.81746, validation loss: 1.84363, learning rate: [0.0019869444917922276]
[06/25 15:59:06 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([ 0.1461,  0.1363, -0.0527, -0.0271], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-0.2853, -0.1909, -0.3706,  0.4672], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([1.1911e-43], device='cuda:0', requires_grad=True)]
[06/25 15:59:06 nl.defaults.trainer]: Epoch 87 done. Train accuracy (top1, top5): 61.47143, 94.50857, Validation accuracy: 60.23038, 94.46567
[06/25 15:59:06 nl.optimizers.oneshot.darts.opt

[06/25 16:01:03 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.159551, +0.139306, -0.049809, -0.038293, 0
-0.287383, -0.198598, -0.393645, +0.484081, 3
+0.000000, 0
[06/25 16:01:08 nl.defaults.trainer]: Epoch 93-28, Train loss: 1.82079, validation loss: 1.86226, learning rate: [0.0012889988567350316]
[06/25 16:01:13 nl.defaults.trainer]: Epoch 93-58, Train loss: 1.83895, validation loss: 1.88437, learning rate: [0.0012889988567350316]
[06/25 16:01:18 nl.defaults.trainer]: Epoch 93-88, Train loss: 1.85728, validation loss: 1.84760, learning rate: [0.0012889988567350316]
[06/25 16:01:23 nl.defaults.trainer]: Epoch 93-117, Train loss: 1.81954, validation loss: 1.85884, learning rate: [0.0012889988567350316]
[06/25 16:01:27 nl.optimizers.oneshot.darts.optimizer]: Arch weights before discretization: [Parameter containing:
tensor([ 0.1611,  0.1403, -0.0495, -0.0400], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([-0.2886, -0

[06/25 16:03:24 nl.defaults.trainer]: Epoch 98 done. Train accuracy (top1, top5): 62.53714, 94.43143, Validation accuracy: 60.39291, 94.36017
[06/25 16:03:24 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.164252, +0.156129, -0.061027, -0.046950, 0
-0.286570, -0.203149, -0.408523, +0.497054, 3
+0.000000, 0
[06/25 16:03:26 nl.defaults.trainer]: Epoch 99-9, Train loss: 1.82262, validation loss: 1.92012, learning rate: [0.0010059212756112208]
[06/25 16:03:31 nl.defaults.trainer]: Epoch 99-39, Train loss: 1.83167, validation loss: 1.80664, learning rate: [0.0010059212756112208]
[06/25 16:03:36 nl.defaults.trainer]: Epoch 99-69, Train loss: 1.84602, validation loss: 1.85187, learning rate: [0.0010059212756112208]
[06/25 16:03:41 nl.defaults.trainer]: Epoch 99-99, Train loss: 1.83823, validation loss: 1.83787, learning rate: [0.0010059212756112208]
[06/25 16:03:46 nl.defaults.trainer]: Epoch 99-128, Train loss: 1.86664, validation loss: 1.83685, learnin

FileNotFoundError: [Errno 2] No such file or directory: 'run/cifar10/darts/0/search/model_final.pth'

In [7]:
trainer.evaluate_oneshot()

[06/25 16:18:16 nl.defaults.trainer]: Start one-shot evaluation
Files already downloaded and verified
Files already downloaded and verified
[06/25 16:18:22 nl.defaults.trainer]: Evaluation finished


60.08148266374351